In [2]:
import requests
import pandas as pd
import requests
import numpy as np
import concurrent.futures


In [5]:
excel_file = r"D:\Onedrive\OneDrive - UNI CONSULTING\Attachments\Code\uni_craw_tvpl\uni_craw_korea\image\new_excel_file_2.xlsx"  # Đường dẫn đến file Excel của bạn

In [6]:
df = pd.read_excel(excel_file)


In [15]:
df

,참조번호,시행일자,결정세번,품명,물품설명,결정사유,이미지건수,관련 이미지,rrdcNo,Uploaded Links
0,Số văn bản tham chiếu và đặt tên file,Ngày văn bản,Mã hàng theo KQ phân loại,Tên hàng khai báo,Tóm tắt mô tả đặc tính hàng hóa,Lý do quyết định,NaN,NaN,NaN,NaN
1,doc_no,doc_date,hs,product_name,product_properties,decision_reason,NaN,NaN,NaN,NaN
2,품목분류2과-6417,2024-08-14,1006.30-1000,"Nonglutinous rice, partially steamed; OM SHREE...",ㅇ 완전히 호화되지 않은 낱알상의 멥쌀을 수지제 봉지에 소매포장한 것(내용량: 5k...,"ㅇ 관세율표 제1006호에서는 “쌀”을 분류하고 있으며, 소호 제1006.30-10...",1.0,['/clip/framework/filedownload/kcs4gImageDownl...,7.202400e+10,['https://uniconsulting079-my.sharepoint.com/p...
3,품목분류2과-5041,2024-07-02,1006.30-1000,"Nonglutinous rice, partially steamed; Steamed ...",완전히 호화되지 않은 낱알상의 멥쌀<br/>- 용도: 식용<br/> <br/>※ 품...,"ㅇ 관세율표 제1006호에 “쌀”을 분류하며, 소호 제1006.30호에 “정미(연마...",1.0,['/clip/framework/filedownload/kcs4gImageDownl...,7.202400e+10,['https://uniconsulting079-my.sharepoint.com/p...
4,품목분류2과-5042,2024-07-02,1006.30-1000,"Nonglutinous rice, partially steamed; Steamed ...",완전히 호화되지 않은 낱알상의 멥쌀<br/>- 용도: 식용<br/> <br/>※ 품...,"ㅇ 관세율표 제1006호에 “쌀”을 분류하며, 소호 제1006.30호에 “정미(연마...",1.0,['/clip/framework/filedownload/kcs4gImageDownl...,7.202400e+10,['https://uniconsulting079-my.sharepoint.com/p...
...,...,...,...,...,...,...,...,...,...,...
27292,품목분류과-102634,2005-01-07,9506.99-0000,ㅇ품명(화주제시) : Link-A-Doos Take Along Swing<br/> ...,□ 물품개요@§ ㅇ사용연령 : 신생아 이상@§ ㅇ구조 및 형태@§ - 철제...,ㅇ제9506호는 일반적인 육체적 운동·체조·육상·기타의 운동용구 또<br/>는 옥외...,0.0,[],NaN,NaN
27293,심사정책과-100046,2006-10-30,9602.00-9090,Konjac Sponge ; 중국산,<br/>ㅇ 성분 및 형태<br/> <br/> - 곤약분을 물과 함께 끓인 액을...,ㅇ 관세율표 제9602호는 “다른 호에 분류되지 아니한 기타의 성형품”<br/>을 ...,0.0,[],NaN,NaN
27294,품목분류과-102634,2005-01-07,9603.50-0000,Anti-Static Brus,"고객의 요구에 따라 여러 가지 형태 및 규격으로 제조되며, 타입은 Tape @§Ty...","ㅇ 고객의 요구에 따라 여러 가지 형태 및 규격으로 제조되며, 타입은 <br/>Ta...",0.0,[],NaN,NaN
27295,품목분류3과-4980,2024-09-24,9701.92-0000,Enamelled Lava Stone Tiles,ㅇ 신안군의 도초도 예술섬 프로젝트에 선정된 ‘올라퍼 엘리아슨’의 “대지의 파빌리온...,- 관세율표 제97류 주 제5호 가목에서 “이 류와 이 표의 다른 류로 동시에 분류...,1.0,['/clip/framework/filedownload/kcs4gImageDownl...,NaN,NaN


In [ ]:
# Hàm tải ảnh từ URL về máy cục bộ
def download_image(url):
    try:
        # Lấy tên file từ URL
        url = url.strip("'\"")  # Loại bỏ ký tự thừa ở đầu và cuối
        file_name = url.split("attchFileId=")[-1]  # Trích xuất ID từ URL
        file_name = f"{file_name}.jpg"  # Thêm phần mở rộng file

        response = requests.get(url, stream=True)
        response.raise_for_status()  # Kiểm tra lỗi HTTP

        with open(file_name, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print("Đã tải ", file_name)
        return file_name
    except Exception as e:
        print(f"Lỗi tải ảnh: {e}")
        return None

# Hàm tải ảnh và cập nhật `Uploaded Links` cho mỗi hàng
def process_row(index, row):
    uploaded_links = []
    
    if row[6] >= 1 and (pd.isna(row[9]) or not row[9] or "lỗi" in row[9]):
        image_links = row[7]
        
        if pd.notna(image_links):  # Kiểm tra ô không rỗng
            image_links = image_links.strip('][').split(', ')
            if image_links == ['']:
                return index, ''  # Nếu không có link ảnh trong hàng đó
            else:
                for link in image_links:
                    try:
                        link = link.strip("'")
                        full_path_link = "https://unipass.customs.go.kr" + link
                        local_filename = download_image(full_path_link)  # Hàm tải ảnh về
                        onlinepath = "https://yourdomain.com/uploads/"  # Thay bằng đường dẫn chính xác
                        if local_filename:
                            uploaded_links.append(onlinepath + local_filename)
                        else:
                            uploaded_links.append("lỗi")  # Đánh dấu lỗi vào danh sách nếu gặp lỗi
                    except Exception as e:
                        print(f"Lỗi khi tải ảnh từ {link}: {e}")
                        uploaded_links.append("lỗi")  # Đánh dấu lỗi vào danh sách nếu gặp lỗi

                return index, ", ".join(uploaded_links)  # Gộp các liên kết đã tải lên thành chuỗi

    return index, "lỗi"  # Nếu `image_links` rỗng hoặc không tải được ảnh

# Hàm để xử lý mỗi phần của DataFrame với các hàng nhất định
def process_dataframe_part(part_df):
    results = []
    for index, row in part_df.iterrows():
        result = process_row(index, row)
        results.append(result)
    return results

# Giả sử df là DataFrame của bạn
# Chia DataFrame thành 50 phần nhỏ
num_threads = 50
df_split = np.array_split(df, num_threads)

# Khởi tạo ThreadPoolExecutor với 50 thread
with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Xử lý từng phần của DataFrame song song
    futures = [executor.submit(process_dataframe_part, part_df) for part_df in df_split]

    # Cập nhật lại DataFrame với kết quả đã xử lý
    for future in concurrent.futures.as_completed(futures):
        results = future.result()
        for index, uploaded_links in results:
            df.at[index, 'Uploaded Links'] = uploaded_links  # Cập nhật lại cột 'Uploaded Links'

print("Hoàn tất xử lý đa luồng.")


c:\Users\nguye\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
C:\Users\nguye\AppData\Local\Temp\ipykernel_9904\3990151690.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[6] >= 1 and (pd.isna(row[9]) or not row[9] or "lỗi" in row[9]):
C:\Users\nguye\AppData\Local\Temp\ipykernel_9904\3990151690.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_links = row[7]


Đã tải  PCA-20170217-000012762967n1I9.jpg
Đã tải  FILEIMG_PCA004M0000010336.jpg
Đã tải  PCA-20161229-00001190696TVvFA.jpg
Đã tải  PCA-20211231-00004546449n1GWc.jpg
Đã tải  PCA-20161229-00001187835nwsYt.jpg
Đã tải  PCA-20161229-00001188597eKpFU.jpg
Đã tải  PCA-20180404-00002249094OkUUr.jpg
Đã tải  PCA-20170206-00001256968UySFe.jpg
Đã tải  PCA-20161229-00001187445KNaih.jpg
Đã tải  FILEIMG_PCA004M0000004055.jpg
Đã tải  FILEIMG_PCA004M0000012166.jpg
Đã tải  PCA-20161229-00001187868qug0D.jpg
Đã tải  PCA-20170522-00001454253Ap5pH.jpg
Đã tải  PCA-20221201-000050210236YrZk.jpg
Đã tải  PCA-20180604-00002357948bUfgW.jpg
Đã tải  PCA-20200507-00003598755xdzTm.jpg
Đã tải  FILEIMG_PCA004M0000007647.jpg
Đã tải  PCA-20171204-00001805134pjQlj.jpg
Đã tải  FILEIMG_PCA004M0000002675.jpg
Đã tải  PCA-20240318-00005637228iOn0I.jpg
Đã tải  PCA-20210119-00004030603rwA0N.jpg
Đã tải  PCA-20211117-0000447314562lZK.jpg
Đã tải  PCA-20161229-000011876655IvHO.jpg
Đã tải  PCA-20220715-00004837904aVViW.jpg
Đã tải  PCA-

In [23]:
len(df_split)

50

In [1]:
df

NameError: name 'df' is not defined

In [ ]:
# Ghi lại DataFrame vào file Excel
df.to_excel(r"D:\Onedrive\OneDrive - UNI CONSULTING\Attachments\Code\uni_craw_tvpl\uni_craw_korea\threads\new_excel_file_2.xlsx", index=False)